# Where to put a 99-Ranch or H-Mart in San Antonio?

by Andrew Metzger

San Antonio, despite being the 7th most populous city in the country and the 2nd most populous city in the state of Texas, has no "China town". According to the U.S. Census, San Antonio also has a similar Asian population to Dallas (45k vs 50k) which has its own Asian food district. Even Katy, TX has a "China town" with its Asian population of 867 (2019 US Census). \
San Antonio does, however, have some smaller grocery stores scattered about the city but no mainstream Asian stores such as H-Mart or 99-Ranch. A medium-sized Asia Super Market opened on the East Side of town, but its debut was met with the COVID pandemic. Most seekers of East Asian cuisine and ingredients have to drive hours to Austin or Houston, TX. My goal for this project is to find an ideal part of San Antonio to open/build a large Asian market in hopes to nucleate a "China town" and increase the cultural diversity of the city. This will probably involve comparing Zip codes' Asian populations and their number of Asian restaurants. Beneficiaries of this research will be the Asian communities and those who appreciate Asian culture and cuisine.

#### An Asian supermarket probably needs to be built where many Asian people are. We can get an idea of the distribution of Asian people in San Antonio from the US census. 

But first let's import the necessary packages...

In [177]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import ssl

### Now, let's get the ZIP code (ZCTA5) demographics for Texas (downloaded from data.census.gov).

In [178]:
demo_df = pd.read_csv('zip_code_demo_2019.csv', skiprows =1)
demo_df.head()

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,...,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,8600000US75001,ZCTA5 75001,14992,219,8211,605,2194,381,115,92,...,0,21,1800,603,1177,276,183,104,994,259
1,8600000US75002,ZCTA5 75002,71253,1424,52482,1775,7566,954,299,153,...,0,32,1308,598,2396,649,261,149,2135,635
2,8600000US75006,ZCTA5 75006,51642,547,40058,993,4590,597,80,54,...,44,46,1955,571,1060,310,169,87,891,295
3,8600000US75007,ZCTA5 75007,55500,1303,38046,1339,5078,775,279,145,...,90,104,1574,506,1516,353,359,153,1157,331
4,8600000US75009,ZCTA5 75009,14089,1262,12559,1691,1006,871,37,40,...,0,21,191,163,250,161,33,69,217,154


#### What are the names of the columns?

In [179]:
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935 entries, 0 to 1934
Data columns (total 22 columns):
 #   Column                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                     --------------  ----- 
 0   id                                                                                                         1935 non-null   object
 1   Geographic Area Name                                                                                       1935 non-null   object
 2   Estimate!!Total:                                                                                           1935 non-null   int64 
 3   Margin of Error!!Total:                                                                                    1935 non-null   object
 4   Estimate!!Total:!!White alone                                                 

In [180]:
#Change ZCTA5 ##### to ##### Zipcodes
demo_df['Geographic Area Name'] = demo_df['Geographic Area Name'].str.replace("ZCTA5 ","")
#Change column names
demo_df = demo_df.rename(columns = {'Margin of Error!!Total:!!Asian alone':'Est_Asian', 'Geographic Area Name':'Zip_Code', 
                          'Estimate!!Total:':'Est_Total','Estimate!!Total:!!White alone':'Est_White','Estimate!!Total:!!Black or African American alone':'Est_Black', 
                          'Estimate!!Total:!!American Indian and Alaska Native alone':'Est_Ind_Alaskan_Native','Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone':'Est_Hawaiian_PI',
                         'Estimate!!Total:!!Some other race alone':'Est_other', 'Estimate!!Total:!!Two or more races:': 'Est_Multiracial',
                         'Estimate!!Total:!!Two or more races:!!Two races including Some other race':'Est_two_other','Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races':'Est_two_other_three'})
demo_df.head()

,id,Zip_Code,Est_Total,Margin of Error!!Total:,Est_White,Margin of Error!!Total:!!White alone,Est_Black,Margin of Error!!Total:!!Black or African American alone,Est_Ind_Alaskan_Native,Margin of Error!!Total:!!American Indian and Alaska Native alone,...,Est_Hawaiian_PI,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Est_other,Margin of Error!!Total:!!Some other race alone,Est_Multiracial,Margin of Error!!Total:!!Two or more races:,Est_two_other,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Est_two_other_three,"Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,8600000US75001,75001,14992,219,8211,605,2194,381,115,92,...,0,21,1800,603,1177,276,183,104,994,259
1,8600000US75002,75002,71253,1424,52482,1775,7566,954,299,153,...,0,32,1308,598,2396,649,261,149,2135,635
2,8600000US75006,75006,51642,547,40058,993,4590,597,80,54,...,44,46,1955,571,1060,310,169,87,891,295
3,8600000US75007,75007,55500,1303,38046,1339,5078,775,279,145,...,90,104,1574,506,1516,353,359,153,1157,331
4,8600000US75009,75009,14089,1262,12559,1691,1006,871,37,40,...,0,21,191,163,250,161,33,69,217,154


#### Now let's get the coordinates for the zip codes for San Antonio. You can get this information (for the USA) from http://download.geonames.org/export/zip/.

In [181]:
us_zip = pd.read_csv('US.txt', delimiter = '\t', header = None )
us_zip.rename(columns = {0:'Country', 1:'ZIP Code', 2:'Region Name', 3:'State', 4:'State Abbreviation', 5:'County', 9:'Latitude', 10:'Longitude' }, inplace = True)
us_zip.drop(['Country', 6, 7, 8, 11], axis = 1, inplace = True)
#filter for ZIP codes in San Antonio, Texas
tx_zip = us_zip.loc[(us_zip['Region Name'] == 'San Antonio') & (us_zip['State'] == 'Texas')]
tx_zip.head()

,ZIP Code,Region Name,State,State Abbreviation,County,Latitude,Longitude
33958,78201,San Antonio,Texas,TX,Bexar,29.4711,-98.5356
33959,78202,San Antonio,Texas,TX,Bexar,29.4275,-98.4601
33960,78203,San Antonio,Texas,TX,Bexar,29.4148,-98.4601
33961,78204,San Antonio,Texas,TX,Bexar,29.4059,-98.5078
33962,78205,San Antonio,Texas,TX,Bexar,29.4237,-98.4925


#### Now let's merge the SA zip data with the locational data...

In [182]:
tx_zip['ZIP Code'] = tx_zip['ZIP Code'].astype(str)
df_combined = demo_df.merge(tx_zip, how = 'outer', left_on = 'Zip_Code', right_on = 'ZIP Code')
#drop Zip codes not in San Antonio (those with null values)
#df_combined.dropna(subset = ['Latitude'], inplace = True)
df_combined.reset_index(drop=True)
df_combined.head()

/usr/local/Cellar/jupyterlab/3.0.16_1/libexec/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,Zip_Code,Est_Total,Margin of Error!!Total:,Est_White,Margin of Error!!Total:!!White alone,Est_Black,Margin of Error!!Total:!!Black or African American alone,Est_Ind_Alaskan_Native,Margin of Error!!Total:!!American Indian and Alaska Native alone,...,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,Est_two_other_three,"Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races",ZIP Code,Region Name,State,State Abbreviation,County,Latitude,Longitude
0,8600000US75001,75001,14992.0,219,8211.0,605.0,2194.0,381.0,115.0,92.0,...,104.0,994.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8600000US75002,75002,71253.0,1424,52482.0,1775.0,7566.0,954.0,299.0,153.0,...,149.0,2135.0,635.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8600000US75006,75006,51642.0,547,40058.0,993.0,4590.0,597.0,80.0,54.0,...,87.0,891.0,295.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8600000US75007,75007,55500.0,1303,38046.0,1339.0,5078.0,775.0,279.0,145.0,...,153.0,1157.0,331.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8600000US75009,75009,14089.0,1262,12559.0,1691.0,1006.0,871.0,37.0,40.0,...,69.0,217.0,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
df_combined.dropna(subset = ['Latitude'], inplace = True)
df_combined.shape

(83, 29)

29 columns is a lot to handle.
#### Let's make a df that has the Asian population, latitude, longitude and Zip code


In [184]:
df_azn = df_combined[['ZIP Code', 'Est_Asian', 'Est_Hawaiian_PI', 'Est_Total','Latitude', 'Longitude']].reset_index(drop = True)
df_azn.head()

,ZIP Code,Est_Asian,Est_Hawaiian_PI,Est_Total,Latitude,Longitude
0,78201,147.0,0.0,48334.0,29.4711,-98.5356
1,78202,51.0,5.0,11453.0,29.4275,-98.4601
2,78203,30.0,0.0,6180.0,29.4148,-98.4601
3,78204,54.0,0.0,12231.0,29.4059,-98.5078
4,78205,14.0,0.0,1441.0,29.4237,-98.4925


Drop any null values in the population (some ZIP codes are post office boxes). Calculate Asian American Pacific Islander pop.

In [185]:
df_azn.dropna(subset = ['Est_Asian'], inplace = True)
df_azn['Est_AAPI'] = df_azn['Est_Asian']+df_azn['Est_Hawaiian_PI']
df_azn.sort_values('Est_AAPI', ascending = False)

,ZIP Code,Est_Asian,Est_Hawaiian_PI,Est_Total,Latitude,Longitude,Est_AAPI
36,78240,1487.0,140.0,60927.0,29.5189,-98.6006,1627.0
52,78258,856.0,0.0,44747.0,29.6562,-98.4967,856.0
44,78250,676.0,160.0,61669.0,29.5054,-98.6688,836.0
48,78254,595.0,201.0,59107.0,29.5551,-98.7442,796.0
45,78251,719.0,47.0,62186.0,29.4597,-98.6555,766.0
43,78249,609.0,59.0,61639.0,29.5612,-98.6117,668.0
40,78245,424.0,226.0,76518.0,29.4189,-98.6895,650.0
28,78230,631.0,0.0,45021.0,29.5407,-98.5521,631.0
41,78247,541.0,44.0,53892.0,29.5855,-98.4071,585.0
19,78221,491.0,31.0,41185.0,29.3309,-98.5054,522.0


#### 78240 seems to have the largest number of Asians in the city. Hawaiian and Pacific Islander can also be included in the totals (AAPI or Asian American Pacific Islander). What about percent of total pop?

In [186]:
#percent AAPI
df_azn['Percent Asian'] = (df_azn['Est_Asian']+df_azn['Est_Hawaiian_PI'])/df_azn['Est_Total']
#AAPI population for the ZIP codes

df_azn.sort_values('Percent Asian', ascending = False).head()

,ZIP Code,Est_Asian,Est_Hawaiian_PI,Est_Total,Latitude,Longitude,Est_AAPI,Percent Asian
38,78243,58.0,24.0,408.0,29.4375,-98.4616,82.0,0.200980
50,78256,333.0,47.0,11238.0,29.6169,-98.6252,380.0,0.033814
58,78266,210.0,14.0,6913.0,29.6643,-98.3118,224.0,0.032403
51,78257,222.0,0.0,7538.0,29.6495,-98.6137,222.0,0.029451
36,78240,1487.0,140.0,60927.0,29.5189,-98.6006,1627.0,0.026704


 #### 78240 is still in the top 5 in terms of Asians as percent of the total Zip code population, 
 but the other 4 Zip codes with higher percentages have less Asian people combined than 78240. Interestingly, 78243 has a large percentage of Asian and Pacific Islanders as well. 78243 is part of Lackland Airforce base, and 78240 is adjacent to the Medical Center.

#### In terms of customer base, 78240 is a good zip code. The number of Asian restaurants in each zip code may be another good metric.
First, let's see what kind of venues are in each zip code and cluster the zip codes.

In [30]:
CLIENT_ID = 'GRQDMKG5QXBA34GNMLFNMKLR401CT113G5GY253GY1PNUWVK' # your Foursquare ID
CLIENT_SECRET = 'LZTGZJ5SDZHMAV22XHGOG3HJEFDAAXCE3H5CXEOORJDT11AS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Function for obtaining venues:

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get data from Foursquare on Venues in San Antonio

In [32]:
sa_venues = getNearbyVenues(names=df_azn['ZIP Code'],
                                   latitudes=df_azn['Latitude'],
                                   longitudes=df_azn['Longitude'])

78201
78202
78203
78204
78205
78207
78208
78209
78210
78211
78212
78213
78214
78215
78216
78217
78218
78219
78220
78221
78222
78223
78224
78225
78226
78227
78228
78229
78230
78231
78232
78233
78235
78237
78238
78239
78240
78242
78243
78244
78245
78247
78248
78249
78250
78251
78252
78253
78254
78255
78256
78257
78258
78259
78260
78261
78263
78264
78266


In [33]:
sa_venues.head()

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,78201,29.4711,-98.5356,Original Donut Shop,29.472703,-98.534598,Donut Shop
1,78201,29.4711,-98.5356,Restaurant Depot,29.473163,-98.535505,Kitchen Supply Store
2,78201,29.4711,-98.5356,Walgreens,29.473850,-98.534403,Pharmacy
3,78201,29.4711,-98.5356,De Wese's Tip Top Cafe,29.470214,-98.531357,Café
4,78201,29.4711,-98.5356,Tink-A-Tako,29.477054,-98.539218,Mexican Restaurant


How many venue types?

In [34]:
print('There are {} uniques categories.'.format(len(sa_venues['Venue Category'].unique())))

There are 221 uniques categories.


### Next we'll see what kind of venues are in each ZIP code

In [35]:
# one hot encoding
sa_onehot = pd.get_dummies(sa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sa_onehot['Zip Code'] = sa_venues['Zip Code'] 

# move neighborhood column to the first column
sa_onehot = sa_onehot[ ['Zip Code'] + [ col for col in sa_onehot.columns if col != 'Zip Code' ] ]

sa_onehot.head()

,Zip Code,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,78201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,78201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,78201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,78201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78201,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


How many venues did we obtain?

In [36]:
sa_onehot.shape

(1161, 222)

In [126]:
#count number of each venue type per zip code
sa_sum = sa_onehot.groupby('Zip Code').sum().reset_index()
sa_sum

,Zip Code,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,78201,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,78202,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,78203,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,78204,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,78205,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
5,78207,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,78208,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,78209,0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,78210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,78211,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
#group the venue types by ZIP Code
sa_grouped = sa_onehot.groupby('Zip Code').mean().reset_index()
sa_grouped

,Zip Code,Accessories Store,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,78201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,78202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,78203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,78204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,78205,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000
5,78207,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,78208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,78209,0.000000,0.027027,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000
8,78210,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,78211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
len(sa_grouped.columns)

222

#### Now to see if any stand out for Asian Restaurants...

In [40]:
num_top_venues = 5

for hood in sa_grouped['Zip Code']:
    print("----"+hood+"----")
    temp = sa_grouped[sa_grouped['Zip Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----78201----
                  venue  freq
0    Mexican Restaurant  0.22
1     Convenience Store  0.07
2  Kitchen Supply Store  0.04
3             Pet Store  0.04
4             Pawn Shop  0.04


----78202----
                venue  freq
0  Mexican Restaurant  0.12
1           BBQ Joint  0.06
2   Mobile Phone Shop  0.06
3       Grocery Store  0.06
4                Café  0.06


----78203----
                             venue  freq
0               Mexican Restaurant  0.29
1              Fried Chicken Joint  0.14
2  Southern / Soul Food Restaurant  0.14
3                     Home Service  0.14
4                             Park  0.14


----78204----
                    venue  freq
0      Mexican Restaurant  0.28
1                     Bar  0.08
2             Pizza Place  0.08
3  Gluten-free Restaurant  0.04
4                 Brewery  0.04


----78205----
                venue  freq
0               Hotel  0.15
1  Mexican Restaurant  0.06
2             Theater  0.05
3  Seafood Restaurant  0

 So in terms of frequency between venues, 78222 and 78229 have a few Chinese restaurants, and 78259 has a few Thai Restaurants. 

#### Let's try explicitly searching with Foursquare for Asian restaurants.

In [108]:
def getNearbyAsianRest(names, latitudes, longitudes, radius=1200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL category id is for 'Asian Restaurants'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d142941735'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']
    ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                            ]
    
    return(nearby_venues)

In [109]:
#call the modified function
sa_rest = getNearbyAsianRest(names=df_azn['ZIP Code'],
                                   latitudes=df_azn['Latitude'],
                                   longitudes=df_azn['Longitude'])

78201
78202
78203
78204
78205
78207
78208
78209
78210
78211
78212
78213
78214
78215
78216
78217
78218
78219
78220
78221
78222
78223
78224
78225
78226
78227
78228
78229
78230
78231
78232
78233
78235
78237
78238
78239
78240
78242
78243
78244
78245
78247
78248
78249
78250
78251
78252
78253
78254
78255
78256
78257
78258
78259
78260
78261
78263
78264
78266


### How many Asian Restaurants are their in each ZIP code?

In [110]:
sa_rest_count= sa_rest.groupby('Zip Code').count().sort_values('Venue', ascending = False)
sa_rest_count['Venue']

Zip Code
78229    18
78205    18
78212    13
78218    11
78251     9
78240     9
78227     9
78215     8
78217     7
78216     6
78213     6
78232     4
78249     4
78214     4
78259     4
78204     4
78223     4
78235     3
78237     3
78243     3
78208     3
78202     2
78222     2
78230     2
78248     2
78233     2
78203     2
78201     2
78250     1
78257     1
78244     1
78226     1
78239     1
78238     1
78231     1
78225     1
78224     1
78220     1
78209     1
78207     1
78260     1
Name: Venue, dtype: int64

 
 Looks like from the last analysis, 78229 has the most number of Asian restaurants (within 2 km). 78240 is in 12th place.

#### How close are all of these Zip codes to each other?

In [111]:
df_mapping = sa_rest_count.merge(df_azn, how = 'outer', left_on = 'Zip Code', right_on = 'ZIP Code')
df_mapping.drop(axis = 1, labels = ['Zip Code Latitude', 'Zip Code Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], inplace =True)
df_mapping

,Venue,ZIP Code,Est_Asian,Est_Hawaiian_PI,Est_Total,Latitude,Longitude,Percent Asian,Est_AAPI
0,18.0,78229,417.0,47.0,34125.0,29.5042,-98.5697,0.013597,464.0
1,18.0,78205,14.0,0.0,1441.0,29.4237,-98.4925,0.009715,14.0
2,13.0,78212,131.0,37.0,28865.0,29.4388,-98.4935,0.005820,168.0
3,11.0,78218,467.0,3.0,37281.0,29.4969,-98.4032,0.012607,470.0
4,9.0,78251,719.0,47.0,62186.0,29.4597,-98.6555,0.012318,766.0
5,9.0,78240,1487.0,140.0,60927.0,29.5189,-98.6006,0.026704,1627.0
6,9.0,78227,246.0,26.0,51394.0,29.4027,-98.6433,0.005292,272.0
7,8.0,78215,61.0,0.0,2915.0,29.4413,-98.4793,0.020926,61.0
8,7.0,78217,392.0,41.0,34159.0,29.5395,-98.4194,0.012676,433.0
9,6.0,78216,417.0,17.0,43554.0,29.5334,-98.4975,0.009965,434.0


In [112]:
#drop nulls
df_mapping.dropna(axis = 0, how = 'any', inplace = True)

In [122]:
# function to map the ZIP codes, putting bubbles sized by number of Asian Restaurants
import folium
sa_coordinates = [df_mapping['Latitude'].mean(), df_mapping['Longitude'].mean()]
m = folium.Map(location=sa_coordinates, tiles="OpenStreetMap", zoom_start=11)
for i in range(0,len(df_mapping)):
   folium.Circle(
      location=[df_mapping.iloc[i]['Latitude'], df_mapping.iloc[i]['Longitude']],
      popup=df_mapping.iloc[i]['ZIP Code'],
      radius=float(df_mapping.iloc[i]['Venue'])*50,
      color='blue',
      fill=False,
      fill_color='blue'
   ).add_to(m)

m

#### Two areas seem to have a higher density of Asian Restaurants: Downtown and the Medical Center to the Northwest.

Let's take a closer look at the downtown ZIP codes...

In [104]:
sa_rest.loc[sa_rest['Zip Code'] == '78212']

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
33,78212,29.4388,-98.4935,Saigon Express,29.445055,-98.491520,Vietnamese Restaurant
34,78212,29.4388,-98.4935,Pinch Boil House and Bia Bar,29.425906,-98.493674,Seafood Restaurant
35,78212,29.4388,-98.4935,Best Quality Daughter,29.441435,-98.479240,Chinese Restaurant
36,78212,29.4388,-98.4935,Tenko,29.442165,-98.480469,Ramen Restaurant
37,78212,29.4388,-98.4935,Shiro,29.436195,-98.481240,Japanese Restaurant
38,78212,29.4388,-98.4935,Curry Boys Bbq,29.448236,-98.487560,Asian Restaurant
39,78212,29.4388,-98.4935,Sze-Chuen Chinese Restaurant,29.426003,-98.493467,Chinese Restaurant
40,78212,29.4388,-98.4935,Pho Bo Express,29.427437,-98.487520,Vietnamese Restaurant
41,78212,29.4388,-98.4935,Teriyaki Kitchen,29.425472,-98.500511,Chinese Restaurant
42,78212,29.4388,-98.4935,Botika,29.442526,-98.479975,Peruvian Restaurant


In [105]:
sa_rest.loc[sa_rest['Zip Code'] == '78205']

,Zip Code,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
10,78205,29.4237,-98.4925,Sushi Zushi,29.424015,-98.491907,Sushi Restaurant
11,78205,29.4237,-98.4925,Blue Whale,29.423884,-98.493056,Japanese Restaurant
12,78205,29.4237,-98.4925,Pinch Boil House and Bia Bar,29.425906,-98.493674,Seafood Restaurant
13,78205,29.4237,-98.4925,Bamboo,29.413464,-98.499669,Chinese Restaurant
14,78205,29.4237,-98.4925,Sze-Chuen Chinese Restaurant,29.426003,-98.493467,Chinese Restaurant
15,78205,29.4237,-98.4925,Thai Lucky,29.421184,-98.490475,Thai Restaurant
16,78205,29.4237,-98.4925,Side Wok Cafe,29.423517,-98.484732,Asian Restaurant
17,78205,29.4237,-98.4925,Sansei Sushi,29.421921,-98.484116,Sushi Restaurant
18,78205,29.4237,-98.4925,Hot Joy,29.413809,-98.490913,Asian Restaurant
19,78205,29.4237,-98.4925,Pho Bo Express,29.427437,-98.487520,Vietnamese Restaurant


Seems like a few restaurants are shared between 78212 and 78205 (Kimura, Teriyaki Kitchen, Sze-chuan, Pinch Boil House) which are fairly small ZIP codes. This is one of the limits of this analysis - ZIP codes are not all equal area.

#### What if we add circles to the map representing AAPI population for the ZIP codes?

In [123]:
for i in range(0,len(df_mapping)):
   folium.Circle(
      location=[df_mapping.iloc[i]['Latitude'], df_mapping.iloc[i]['Longitude']],
      popup=df_mapping.iloc[i]['ZIP Code'],
      radius=float(df_mapping.iloc[i]['Est_AAPI']),
      color='red',
      fill=False,
      fill_color='red'
   ).add_to(m)
m

With the red circles representing AAPI population, the Northwest side of San Antonio has more AAPI people (likely customers) than Downtown San Antonio. Northwest San Antonio appears to be the best location for a large Asian supermarket, with 78240 having the most AAPI residents and 78229 having the most Asian restaurants.

Let's see if we can find similarities between the ZIP codes in terms of the restaurants collected...

In [130]:
# one hot encoding
rest_onehot = pd.get_dummies(sa_rest[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
rest_onehot['Zip Code'] = sa_rest['Zip Code'] 

# move neighborhood column to the first column
rest_onehot = rest_onehot[ ['Zip Code'] + [ col for col in rest_onehot.columns if col != 'Zip Code' ] ]

rest_onehot.head()

,Zip Code,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Food Service,Food Truck,Hotel Bar,Japanese Restaurant,Korean Restaurant,Noodle House,Peruvian Restaurant,Ramen Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,78201,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,78201,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,78202,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,78202,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,78203,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [132]:
sa_grouped1 = rest_onehot.groupby('Zip Code').mean().reset_index()
sa_grouped1

,Zip Code,Asian Restaurant,Cantonese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Filipino Restaurant,Food Service,Food Truck,Hotel Bar,Japanese Restaurant,Korean Restaurant,Noodle House,Peruvian Restaurant,Ramen Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,78201,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000
1,78202,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,78203,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
3,78204,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000
4,78205,0.277778,0.00,0.277778,0.000000,0.000000,0.000000,0.00,0.055556,0.055556,0.000000,0.055556,0.000000,0.000000,0.055556,0.111111,0.055556,0.055556
5,78207,0.000000,0.00,1.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,78208,0.333333,0.00,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,78209,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
8,78212,0.230769,0.00,0.230769,0.000000,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.076923,0.076923,0.076923,0.076923,0.000000,0.000000,0.153846
9,78213,0.166667,0.00,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.166667,0.000000


Let's take a look at the Asian Restaurants in each Zip code

In [140]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zip_venues_sorted = pd.DataFrame(columns=columns)
zip_venues_sorted['Zip Code'] = sa_grouped1['Zip Code']

for ind in np.arange(sa_grouped1.shape[0]):
    zip_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sa_grouped1.iloc[ind, :], num_top_venues)

zip_venues_sorted.head()

,Zip Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,78201,Vietnamese Restaurant,Chinese Restaurant,Korean Restaurant,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Ramen Restaurant,Peruvian Restaurant,Noodle House,Asian Restaurant
1,78202,Korean Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Seafood Restaurant,Ramen Restaurant,Peruvian Restaurant,Noodle House,Japanese Restaurant
2,78203,Chinese Restaurant,Thai Restaurant,Asian Restaurant,Korean Restaurant,Sushi Restaurant,Seafood Restaurant,Ramen Restaurant,Peruvian Restaurant,Noodle House,Japanese Restaurant
3,78204,Chinese Restaurant,Sushi Restaurant,Ramen Restaurant,Asian Restaurant,Korean Restaurant,Thai Restaurant,Seafood Restaurant,Peruvian Restaurant,Noodle House,Japanese Restaurant
4,78205,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Hotel Bar,Thai Restaurant,Seafood Restaurant,Noodle House,Japanese Restaurant,Vietnamese Restaurant,Cantonese Restaurant


In [134]:
num_top_venues = 5

for hood in sa_grouped1['Zip Code']:
    print("----"+hood+"----")
    temp = sa_grouped1[sa_grouped1['Zip Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----78201----
                   venue  freq
0  Vietnamese Restaurant   0.5
1     Chinese Restaurant   0.5
2      Korean Restaurant   0.0
3        Thai Restaurant   0.0
4       Sushi Restaurant   0.0


----78202----
                venue  freq
0   Korean Restaurant   0.5
1  Chinese Restaurant   0.5
2    Asian Restaurant   0.0
3     Thai Restaurant   0.0
4    Sushi Restaurant   0.0


----78203----
                venue  freq
0  Chinese Restaurant   0.5
1     Thai Restaurant   0.5
2    Asian Restaurant   0.0
3   Korean Restaurant   0.0
4    Sushi Restaurant   0.0


----78204----
                venue  freq
0  Chinese Restaurant  0.50
1    Sushi Restaurant  0.25
2    Ramen Restaurant  0.25
3    Asian Restaurant  0.00
4   Korean Restaurant  0.00


----78205----
                venue  freq
0    Asian Restaurant  0.28
1  Chinese Restaurant  0.28
2    Sushi Restaurant  0.11
3           Hotel Bar  0.06
4     Thai Restaurant  0.06


----78207----
                venue  freq
0  Chinese Restauran

Some ZIP codes have no Asian restaurants, some have quite the diversity.

Let's prepare the data for clustering.

In [135]:
#Function for returning the most common venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now to cluster the ZIP codes...

In [155]:
# set number of clusters
kclusters = 5

rest_grouped_clustering = sa_grouped1.drop('Zip Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rest_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<ipython-input-155-3fb3188808e7>:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



array([0, 0, 0, 0, 0, 1, 0, 2, 0, 0], dtype=int32)

In [172]:
#sometimes the Cluster Labels column needs to be dropped...
#zip_venues_sorted.drop(axis =1, labels = 'Cluster Labels', inplace = True)

In [169]:
# add clustering labels
zip_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
sa_merged = df_mapping
# merge sa_grouped with df_mapping (sa data) to add latitude/longitude for each neighborhood
sa_merged = sa_merged.merge(zip_venues_sorted, how = 'outer', left_on = 'ZIP Code', right_on = 'Zip Code')
sa_merged.head() # check the last columns!

,Venue,ZIP Code,Est_Asian,Est_Hawaiian_PI,Est_Total,Latitude,Longitude,Percent Asian,Est_AAPI,Cluster Labels,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,18.0,78229,417.0,47.0,34125.0,29.5042,-98.5697,0.013597,464.0,0,...,Chinese Restaurant,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Korean Restaurant,Sushi Restaurant,Japanese Restaurant,Filipino Restaurant,Food Service,Food Truck
1,18.0,78205,14.0,0.0,1441.0,29.4237,-98.4925,0.009715,14.0,0,...,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Hotel Bar,Thai Restaurant,Seafood Restaurant,Noodle House,Japanese Restaurant,Vietnamese Restaurant,Cantonese Restaurant
2,13.0,78212,131.0,37.0,28865.0,29.4388,-98.4935,0.005820,168.0,0,...,Asian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Noodle House,Peruvian Restaurant,Ramen Restaurant,Seafood Restaurant,Japanese Restaurant,Dim Sum Restaurant,Filipino Restaurant
3,11.0,78218,467.0,3.0,37281.0,29.4969,-98.4032,0.012607,470.0,0,...,Korean Restaurant,Asian Restaurant,Vietnamese Restaurant,Food Service,Dim Sum Restaurant,Filipino Restaurant,Food Truck,Hotel Bar,Cantonese Restaurant,Chinese Restaurant
4,9.0,78251,719.0,47.0,62186.0,29.4597,-98.6555,0.012318,766.0,0,...,Vietnamese Restaurant,Chinese Restaurant,Thai Restaurant,Filipino Restaurant,Asian Restaurant,Dim Sum Restaurant,Food Service,Food Truck,Hotel Bar,Cantonese Restaurant


Now plot the clusters on the map!

In [171]:
# Latitude of San Antonio? Try mean of the lat/lng table?
latitude = df_mapping['Latitude'].mean()
longitude = df_mapping['Longitude'].mean()
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sa_merged['Latitude'], sa_merged['Longitude'], sa_merged['ZIP Code'], sa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Interestingly, the North side of San Antonio is fairly similar to Downtown in terms of the kinds of Asian Restaurants, at least according to the K Means fit. This does not affect the conclusion that Northwest San Antonio is a good area for opening a large Asian supermarket. 


In [173]:
#the map we made earlier
#size of blue bubble related to # of Asian restaurants
#size of red bubbles related to # of AAPI residents
m